# My Submission: Part Two

The below code is from <b>Part One</b>:

https://github.com/elunatik/Coursera_Capstone/blob/master/Python%20Notebook/My%20Submission%20-%20Part%20One.ipynb

In [1]:
# Importing all necessary libraries from the "Segmenting and Clustering Neighborhoods in New York City" lab
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Importing the Web Scraping libraries
# Install these into your system/environment if not downloaded
import requests
import lxml.html as lh

# The url for the wiki page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

# Removing the '\n' from the last column since this character was in every data cell.
df[col[2][0]] = df[col[2][0]].astype(str).str[:-1]

# Changed the Column Names and removing the 'Not assigned' rows
df.columns = ['PostalCode', 'Borough','Neighborhood']
df = df[df.Borough != 'Not assigned']

# Grouping the dataframe by PostalCode AND Borough then Joining the Neighborhood by a comma
df.set_index(['PostalCode', 'Borough'],inplace=True)
df = df.groupby(level=['PostalCode','Borough'], sort=False).agg( ', '.join)

# Resetting the index back to integers
df.reset_index(inplace=True)

print("The shape of the dataframe is: " + str(df.shape))
df.head()

The shape of the dataframe is: (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


### Part Two: We will now get the Latitude and Longtitude
I tried to use the Geocoder library, but I was not able to get any coordinates. Instead, I used the csv file and extracted the coordinates.

I have Jupyter Notebooks set up on my PC, thus I had to download the csv file then place it in the same folder as my notebooks.

In [2]:
# Reading the csv file, which is in the smae folder as my notebook
geoData = pd.read_csv('Geospatial_Coordinates.csv')

In [3]:
# Using .head() in order to see how the data looks like. In this case it is pretty straightforward
geoData.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [4]:
# Changing the index to postal code to easily group/merge the two datframes based on the left('df') dataframe.
df.set_index('PostalCode', inplace = True) # Left Dataframe
geoData.set_index('Postal Code', inplace = True) # Right Dataframe


In [5]:
# I use .merge() based on the index and with a left join. 
# Left since we want to keep all of the rows from our original dataframe.
# Luckily there are coordinates for each Postal Code.
dfComplete = df.merge(geoData, left_index=True, right_index=True, how='left')
dfComplete.reset_index(inplace=True)
dfComplete.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494


In [6]:
dfComplete.shape

(103, 5)